# Particionamento e Bucketing

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import * 

spark = SparkSession.builder.master('local[*]').getOrCreate()

23/09/23 11:59:10 WARN Utils: Your hostname, elvin-Aspire-E1-571 resolves to a loopback address: 127.0.1.1; using 192.168.190.183 instead (on interface wlp3s0)
23/09/23 11:59:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/23 11:59:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [ ]:
spark.sql("create database desp")
spark.sql("use database desp")

desp_schema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
desp = spark.read.csv("/home/elvin/Documents/Engenharia-De-Dados/Cursos/Formação Spark com Pyspark - Udemy/04. Spark SQL/despachantes.csv", header=False, schema=desp_schema)

desp.write.saveAsTable("Despachantes")

In [6]:
spark.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|     desp|despachantes|      false|
+---------+------------+-----------+



In [7]:
churn = spark.read.csv("/home/elvin/download/Churn.csv", header=True, inferSchema=True, sep=";")

In [8]:
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

In [9]:
churn.write.partitionBy("Geography").saveAsTable("Churn_geo")

In [10]:
spark.sql("select * from Churn_geo").show()

+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|CreditScore|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|Geography|
+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|        619|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|   France|
|        502|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|   France|
|        699|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|   France|
|        822|  Male| 50|     7|       0|            2|        1|             1|         100628|     0|   France|
|        501|  Male| 44|     4|14205107|            2|        0|             1|         749405|     0|   France|
|        684|  Male| 27|     2|13460388|            1|        1|             1|        7172573| 

In [12]:
churn.write.bucketBy(3, "Geography").saveAsTable("Churn_geo2")

In [13]:
spark.sql("select * from Churn_geo2").show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

# Cache e Persistência

In [14]:
from pyspark import StorageLevel

In [15]:
spark.sql("use desp")
spark.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|     desp|   churn_geo|      false|
|     desp|  churn_geo2|      false|
|     desp|despachantes|      false|
+---------+------------+-----------+



In [16]:
df = spark.sql("select * from despachantes")
df.show(2)

+---+----------------+------+-------------+------+----------+
| id|            nome|status|       cidade|vendas|      data|
+---+----------------+------+-------------+------+----------+
|  1|Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2| Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
+---+----------------+------+-------------+------+----------+
only showing top 2 rows



In [17]:
df.storageLevel #disco, memória, offheap, serializado, replicação

StorageLevel(False, False, False, False, 1)

In [18]:
df.cache()

DataFrame[id: int, nome: string, status: string, cidade: string, vendas: int, data: string]

In [19]:
df.storageLevel

StorageLevel(True, True, False, True, 1)

In [21]:
df.persist(StorageLevel.DISK_ONLY)

23/09/23 12:40:26 WARN CacheManager: Asked to cache already cached data.


DataFrame[id: int, nome: string, status: string, cidade: string, vendas: int, data: string]

In [22]:
df.unpersist()

DataFrame[id: int, nome: string, status: string, cidade: string, vendas: int, data: string]

In [24]:
df.persist(StorageLevel.DISK_ONLY)

DataFrame[id: int, nome: string, status: string, cidade: string, vendas: int, data: string]

In [25]:
df.storageLevel

StorageLevel(True, False, False, False, 1)